To convert your Python and Streamlit project into a Flutter application with real-time gesture detection, you would typically split it into two components:

FastAPI Backend: Handles the model inference and real-time communication between the client (Flutter app) and the model.

Flutter Frontend: Captures video input and displays the detected gesture, while also handling real-time audio playback and showing the result.

Here’s a basic outline of how you can achieve this:

1. FastAPI Backend for Gesture Detection
We'll set up a FastAPI backend to handle real-time video frame processing. The backend will use the model to predict gestures and return the predictions to the Flutter frontend.

# FastAPI Backend Code:

In [ ]:
import fastapi
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
import os
import io
from pydantic import BaseModel
import asyncio
import threading
import pygame

# Initialize FastAPI app
app = FastAPI()

# Load the model
model = tf.keras.models.load_model("sign_language_model.h5")

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5
)

# Define class names
sign_language_classes = ["", "", "", "", "", "", "", "", "", "", "", "", "", "", "عذرا", "طعام", "مرحبا", "مساعده", "منزل", "انا", "احبك", "لا", "شكرا", "نعم"]

# Function to process landmarks
def process_landmarks(hand_landmarks):
    landmarks = []
    for lm in hand_landmarks.landmark:
        landmarks.extend([lm.x, lm.y, lm.z])
    return landmarks

# Function to classify gestures
def classify_gesture(frame):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)

    if result.multi_hand_landmarks:
        combined_landmarks = []

        # Process first hand
        combined_landmarks.extend(process_landmarks(result.multi_hand_landmarks[0]))

        # Process second hand or pad
        if len(result.multi_hand_landmarks) > 1:
            combined_landmarks.extend(process_landmarks(result.multi_hand_landmarks[1]))

        # Make prediction
        landmarks_array = np.array(combined_landmarks).reshape(1, -1)
        prediction = model.predict(landmarks_array, verbose=0)
        class_id = np.argmax(prediction[0])
        confidence = prediction[0][class_id]

        return sign_language_classes[class_id], confidence

    return None, None

# FastAPI WebSocket endpoint for real-time communication
@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()

    try:
        while True:
            # Receive frame from Flutter app
            frame_data = await websocket.receive_bytes()
            nparr = np.frombuffer(frame_data, np.uint8)
            frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

            # Classify gesture
            gesture, confidence = classify_gesture(frame)

            if gesture:
                await websocket.send_text(f"Gesture: {gesture}, Confidence: {confidence:.2%}")
            else:
                await websocket.send_text("No gesture detected")

    except WebSocketDisconnect:
        print("Client disconnected")


This FastAPI server will listen for WebSocket connections and process incoming frames. It will send back the detected gesture and confidence score to the connected client.

2. Flutter Frontend
The Flutter frontend will capture the video feed, send it to the FastAPI backend via WebSocket, and display the gesture detection result in real-time.

To achieve this:

Add WebSocket support in Flutter: You'll need to use the web_socket_channel Flutter package to communicate with the FastAPI backend over WebSocket.

Capture video in Flutter: You can use the camera package to capture video frames in Flutter.

Send frames to FastAPI: Once you have the video frames, you send them to the FastAPI server and receive the gesture detection result.



# **Flutter Code:**
# **Add dependencies in pubspec.yaml:**

In [ ]:
dependencies:
  flutter:
    sdk: flutter
  web_socket_channel: ^2.1.0
  camera: ^0.9.4+5
  audioplayers: ^0.20.1


# **Flutter App Code:**

In [ ]:
import 'package:flutter/material.dart';
import 'package:web_socket_channel/web_socket_channel.dart';
import 'package:camera/camera.dart';
import 'dart:typed_data';
import 'dart:async';
import 'dart:convert';

void main() {
  runApp(MyApp());
}

class MyApp extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      home: SignLanguageDetection(),
    );
  }
}

class SignLanguageDetection extends StatefulWidget {
  @override
  _SignLanguageDetectionState createState() => _SignLanguageDetectionState();
}

class _SignLanguageDetectionState extends State<SignLanguageDetection> {
  CameraController? _cameraController;
  WebSocketChannel? _channel;
  bool _isCameraInitialized = false;

  @override
  void initState() {
    super.initState();
    _initializeCamera();
    _connectWebSocket();
  }

  Future<void> _initializeCamera() async {
    final cameras = await availableCameras();
    final firstCamera = cameras.first;

    _cameraController = CameraController(firstCamera, ResolutionPreset.medium);
    await _cameraController?.initialize();

    setState(() {
      _isCameraInitialized = true;
    });
  }

  void _connectWebSocket() {
    _channel = WebSocketChannel.connect(Uri.parse("ws://127.0.0.1:8000/ws"));
    _channel?.stream.listen((data) {
      // Handle the message from server
      print(data);
    });
  }

  @override
  void dispose() {
    _cameraController?.dispose();
    _channel?.sink.close();
    super.dispose();
  }

  Future<void> _sendFrameToServer(CameraImage image) async {
    // Convert the frame to a byte array (JPEG/PNG)
    final bytes = await _convertYUV420toJPEG(image);
    _channel?.sink.add(bytes);
  }

  Future<Uint8List> _convertYUV420toJPEG(CameraImage image) async {
    // Convert YUV420 frame to JPEG (You can use the `image` library for this)
    return Uint8List(0); // Replace with actual conversion
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text("Sign Language Detection")),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            _isCameraInitialized
                ? CameraPreview(_cameraController!)
                : CircularProgressIndicator(),
            SizedBox(height: 20),
            ElevatedButton(
              onPressed: () {
                // Start sending frames
                _cameraController?.startImageStream((image) {
                  _sendFrameToServer(image);
                });
              },
              child: Text("Start Detection"),
            ),
          ],
        ),
      ),
    );
  }
}


Key Points:
FastAPI WebSocket: The backend listens for WebSocket connections and processes the frames for gesture detection.

Flutter Camera: The Flutter app captures video frames and sends them to the backend via WebSocket.

WebSocket Communication: The app receives the detected gesture and confidence from the backend and displays it on the screen.

Audio Playback:
For audio playback, you can use Flutter’s audioplayers package or any other package that fits your needs to play the correct audio for the detected gestures.

Let me know if you need further assistance on specific sections or integration!

# **3. Store Audio Files:**

Make sure the audio files (like مرحبا.mp3, نعم.mp3, etc.) are available in your Flutter assets folder. In your pubspec.yaml, add the following:

In [ ]:
flutter:
  assets:
    - assets/audio/


Summary:
The audioplayers package is used for playing the audio corresponding to the detected gesture.

The WebSocket connection sends gesture data, and when a gesture is detected, the correct audio is played.

How it works:
Real-time Gesture Detection:

The Flutter app captures video from the camera using the camera package.

Each frame is sent to the FastAPI backend via WebSocket.

The backend processes the frame, detects the sign language gesture, and sends the gesture name back to the Flutter app.

Audio Playback:

Once the Flutter app receives the gesture name from the backend, it will trigger the corresponding audio file to be played using the audioplayers package.

Steps to Run:
Set up FastAPI:

Ensure that the FastAPI backend is running and listening for WebSocket connections.

The backend will process incoming frames and return detected gestures to the Flutter app.

Set up Flutter:

Ensure the audioplayers package is included and the audio files for each gesture are placed in the assets/audio/ folder.

The Flutter app will capture video, send frames to the backend, and play the corresponding audio when a gesture is detected.

# **Run the Application:**

# **Run the FastAPI server:**

In [ ]:
uvicorn main:app --reload


# **Run the Flutter app:**

In [ ]:
flutter run


Once everything is running:

In real-time, as you make gestures in front of the camera, the system will detect them, and the corresponding audio for that gesture will play.

Things to Check:
Audio Files: Ensure that the audio files for each gesture (e.g., مرحبا.mp3, نعم.mp3, etc.) are available in the assets/audio/ directory and are listed in the pubspec.yaml.

WebSocket Connection: Make sure the Flutter app is connected to the FastAPI WebSocket server. The WebSocket URI in the code is set to ws://127.0.0.1:8000/ws, so it assumes the backend is running locally. If you're running it on a remote server, you need to adjust the WebSocket URI accordingly.